In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/MI_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1752647202,MI,"25800 Northwestern Highway, Suite 720, SOUTHFI...",42.478592,-83.249390,2.612516e+14
1,2139978210,MI,"1 Morley Plaza, SAGINAW, MI",43.434079,-83.938940,2.614500e+14
2,2154087303,MI,"12811 Farmington Road, Livonia, MI",42.377740,-83.372620,2.616356e+14
3,2369917104,MI,"17800 LAUREL PARK DR North, LIVONIA, MI",42.414632,-83.417784,2.616356e+14
4,2844277200,MI,"850 Stephenson Hwy, TROY, MI",42.535515,-83.119950,2.612520e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1752647202,MI,"25800 Northwestern Highway, Suite 720, SOUTHFI...",42.478592,-83.249390,2.612516e+14
1,2139978210,MI,"1 Morley Plaza, SAGINAW, MI",43.434079,-83.938940,2.614500e+14
2,2154087303,MI,"12811 Farmington Road, Livonia, MI",42.377740,-83.372620,2.616356e+14
3,2369917104,MI,"17800 LAUREL PARK DR North, LIVONIA, MI",42.414632,-83.417784,2.616356e+14
4,2844277200,MI,"850 Stephenson Hwy, TROY, MI",42.535515,-83.119950,2.612520e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
765,5652057007,MI,"135 STATE HIGHWAY M35, GWINN, MI",48.024810,-98.124330,NaN
1486,9658427010,MI,"150 W. 2nd Street Suite 300 0.0, Royal Oak, MI",48.025192,-98.123570,NaN
1782,4120198309,MI,"5112 US Highway 31 N, Williamsburg, MI",39.402598,-85.993189,NaN
5467,8715437702,MI,"400 RENAISSANCE CTR STE 2600, DETROIT, MI",42.329117,-83.040093,NaN
5468,8205248602,MI,"5000 N US ROUTE 131, Reed City, MI",43.902312,-85.515920,NaN
5511,9918728402,MI,"3741 Roger B Chaffee Mem Dr SE, Grand Rapids, MI",42.895832,-85.654349,NaN
7108,6885697300,MI,"107 W MICHIGAN AVE STE 100, KALAMAZOO, MI",42.291406,-85.583432,NaN
7112,9074147005,MI,"5000 N US 131, REED CITY, MI",43.902312,-85.515920,NaN
7686,3289778906,MI,"11650 DOWNES ST NE, LOWELL, MI",42.993893,-85.376780,NaN
7776,3168178504,MI,"42211 GARFIELD RD, CLINTON TWP, MI",42.606935,-82.953663,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('26')] #MI
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/45 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

45


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_9849/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
765,5652057007,MI,"135 STATE HIGHWAY M35, GWINN, MI",48.024810,-98.124330,[]
1486,9658427010,MI,"150 W. 2nd Street Suite 300 0.0, Royal Oak, MI",48.025192,-98.123570,[]
1782,4120198309,MI,"5112 US Highway 31 N, Williamsburg, MI",39.402598,-85.993189,[]
5467,8715437702,MI,"400 RENAISSANCE CTR STE 2600, DETROIT, MI",42.329117,-83.040093,[261635172002073]
5468,8205248602,MI,"5000 N US ROUTE 131, Reed City, MI",43.902312,-85.515920,[261339706003002]
5511,9918728402,MI,"3741 Roger B Chaffee Mem Dr SE, Grand Rapids, MI",42.895832,-85.654349,[260810142002006]
7108,6885697300,MI,"107 W MICHIGAN AVE STE 100, KALAMAZOO, MI",42.291406,-85.583432,[260770002013004]
7112,9074147005,MI,"5000 N US 131, REED CITY, MI",43.902312,-85.515920,[261339706003002]
7686,3289778906,MI,"11650 DOWNES ST NE, LOWELL, MI",42.993893,-85.376780,[260810120033005]
7776,3168178504,MI,"42211 GARFIELD RD, CLINTON TWP, MI",42.606935,-82.953663,[260992420011000]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_9849/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
765,5652057007,MI,"135 STATE HIGHWAY M35, GWINN, MI",48.024810,-98.124330,NaN
1486,9658427010,MI,"150 W. 2nd Street Suite 300 0.0, Royal Oak, MI",48.025192,-98.123570,NaN
1782,4120198309,MI,"5112 US Highway 31 N, Williamsburg, MI",39.402598,-85.993189,NaN
5467,8715437702,MI,"400 RENAISSANCE CTR STE 2600, DETROIT, MI",42.329117,-83.040093,261635172002073
5468,8205248602,MI,"5000 N US ROUTE 131, Reed City, MI",43.902312,-85.515920,261339706003002
5511,9918728402,MI,"3741 Roger B Chaffee Mem Dr SE, Grand Rapids, MI",42.895832,-85.654349,260810142002006
7108,6885697300,MI,"107 W MICHIGAN AVE STE 100, KALAMAZOO, MI",42.291406,-85.583432,260770002013004
7112,9074147005,MI,"5000 N US 131, REED CITY, MI",43.902312,-85.515920,261339706003002
7686,3289778906,MI,"11650 DOWNES ST NE, LOWELL, MI",42.993893,-85.376780,260810120033005
7776,3168178504,MI,"42211 GARFIELD RD, CLINTON TWP, MI",42.606935,-82.953663,260992420011000


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
765,5652057007,MI,"135 STATE HIGHWAY M35, GWINN, MI",48.024810,-98.124330,NaN
1486,9658427010,MI,"150 W. 2nd Street Suite 300 0.0, Royal Oak, MI",48.025192,-98.123570,NaN
1782,4120198309,MI,"5112 US Highway 31 N, Williamsburg, MI",39.402598,-85.993189,NaN
10192,5833567104,MI,"666 E MAIN ST Suite A-2, CENTREVILLE, MI",39.958540,-82.981387,NaN
10812,7075478401,MI,"5112 US Highway 31 N, Williamsburg, MI",39.402598,-85.993189,NaN
11779,1919668403,MI,"5665W Hwy US 2, Manistique, MI",48.354809,-114.238664,NaN
11780,2703197109,MI,"5665 W HIGHWAY US 2, MANISTIQUE, MI",48.354809,-114.238664,NaN
12472,3187267103,MI,"9067 US Highway 31, BERRIEN SPRINGS, MI",34.109378,-86.809378,NaN
16050,1405218302,MI,"6279 US Highway 31 S, Charlevoix, MI",39.386384,-85.985616,NaN
16051,8678907004,MI,"06279 US HIGHWAY 31 S, CHARLEVOIX, MI",39.386384,-85.985616,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
765,5652057007,NaN
1486,9658427010,NaN
1782,4120198309,NaN
5467,8715437702,261635172002073
5468,8205248602,261339706003002
5511,9918728402,260810142002006
7108,6885697300,260770002013004
7112,9074147005,261339706003002
7686,3289778906,260810120033005
7776,3168178504,260992420011000


Read in base data

In [15]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/MI_fips_scraped.csv") 
fips_check = fips_check.drop(["Unnamed: 0"], axis =1) 
fips_check.head()

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1752647202,MI,"25800 Northwestern Highway, Suite 720, SOUTHFI...",42.478592,-83.249390,2.612516e+14
1,2139978210,MI,"1 Morley Plaza, SAGINAW, MI",43.434079,-83.938940,2.614500e+14
2,2154087303,MI,"12811 Farmington Road, Livonia, MI",42.377740,-83.372620,2.616356e+14
3,2369917104,MI,"17800 LAUREL PARK DR North, LIVONIA, MI",42.414632,-83.417784,2.616356e+14
4,2844277200,MI,"850 Stephenson Hwy, TROY, MI",42.535515,-83.119950,2.612520e+14


Merge with MainFrame

In [16]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,1752647202,MI,"25800 Northwestern Highway, Suite 720, SOUTHFI...",42.478592,-83.249390,2.612516e+14,NaN
1,2139978210,MI,"1 Morley Plaza, SAGINAW, MI",43.434079,-83.938940,2.614500e+14,NaN
2,2154087303,MI,"12811 Farmington Road, Livonia, MI",42.377740,-83.372620,2.616356e+14,NaN
3,2369917104,MI,"17800 LAUREL PARK DR North, LIVONIA, MI",42.414632,-83.417784,2.616356e+14,NaN
4,2844277200,MI,"850 Stephenson Hwy, TROY, MI",42.535515,-83.119950,2.612520e+14,NaN
...,...,...,...,...,...,...,...
29786,9646208307,MI,"684 Harvey St Ste 201, Muskegon, MI",43.239902,-86.203271,2.612100e+14,NaN
29787,9648118308,MI,"601 Kelso St, Flint, MI",43.020582,-83.651218,2.604900e+14,NaN
29788,9689138310,MI,"22348 Wick Rd, Taylor, MI",42.241625,-83.254483,2.616358e+14,NaN
29789,9788397208,MI,"605 INDUSTRIAL DR, ADRIAN, MI",41.889386,-84.064932,2.609106e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [17]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,1752647202,MI,"25800 Northwestern Highway, Suite 720, SOUTHFI...",42.478592,-83.249390,2.612516e+14,NaN,261251616002002.0
1,2139978210,MI,"1 Morley Plaza, SAGINAW, MI",43.434079,-83.938940,2.614500e+14,NaN,261450004001006.0
2,2154087303,MI,"12811 Farmington Road, Livonia, MI",42.377740,-83.372620,2.616356e+14,NaN,261635583012023.0
3,2369917104,MI,"17800 LAUREL PARK DR North, LIVONIA, MI",42.414632,-83.417784,2.616356e+14,NaN,261635565001037.0
4,2844277200,MI,"850 Stephenson Hwy, TROY, MI",42.535515,-83.119950,2.612520e+14,NaN,261251974002005.0


Drop Extra FIPS Columns

In [19]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1752647202,MI,"25800 Northwestern Highway, Suite 720, SOUTHFI...",42.478592,-83.249390,261251616002002.0
1,2139978210,MI,"1 Morley Plaza, SAGINAW, MI",43.434079,-83.938940,261450004001006.0
2,2154087303,MI,"12811 Farmington Road, Livonia, MI",42.377740,-83.372620,261635583012023.0
3,2369917104,MI,"17800 LAUREL PARK DR North, LIVONIA, MI",42.414632,-83.417784,261635565001037.0
4,2844277200,MI,"850 Stephenson Hwy, TROY, MI",42.535515,-83.119950,261251974002005.0
...,...,...,...,...,...,...
29786,9646208307,MI,"684 Harvey St Ste 201, Muskegon, MI",43.239902,-86.203271,261210001005000.0
29787,9648118308,MI,"601 Kelso St, Flint, MI",43.020582,-83.651218,260490031002006.0
29788,9689138310,MI,"22348 Wick Rd, Taylor, MI",42.241625,-83.254483,261635836002014.0
29789,9788397208,MI,"605 INDUSTRIAL DR, ADRIAN, MI",41.889386,-84.064932,260910617023013.0


In [20]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
765,5652057007,MI,"135 STATE HIGHWAY M35, GWINN, MI",48.024810,-98.124330,NaN
1486,9658427010,MI,"150 W. 2nd Street Suite 300 0.0, Royal Oak, MI",48.025192,-98.123570,NaN
1782,4120198309,MI,"5112 US Highway 31 N, Williamsburg, MI",39.402598,-85.993189,NaN
10192,5833567104,MI,"666 E MAIN ST Suite A-2, CENTREVILLE, MI",39.958540,-82.981387,NaN
10812,7075478401,MI,"5112 US Highway 31 N, Williamsburg, MI",39.402598,-85.993189,NaN
11779,1919668403,MI,"5665W Hwy US 2, Manistique, MI",48.354809,-114.238664,NaN
11780,2703197109,MI,"5665 W HIGHWAY US 2, MANISTIQUE, MI",48.354809,-114.238664,NaN
12472,3187267103,MI,"9067 US Highway 31, BERRIEN SPRINGS, MI",34.109378,-86.809378,NaN
16050,1405218302,MI,"6279 US Highway 31 S, Charlevoix, MI",39.386384,-85.985616,NaN
16051,8678907004,MI,"06279 US HIGHWAY 31 S, CHARLEVOIX, MI",39.386384,-85.985616,NaN


Convert to File

In [21]:
fips_MI = fips_merge_drop

In [22]:
fips_MI.to_csv("../../../data/state_data/geo/geo_fips/23/MI_fips_scraped.csv") 